Importar spark_utils.py

In [ ]:
import sys
sys.path.append('/home/usuario/02025_ESI3914B/spark/libs
') 
from spark_utils import init_spark, load_json_data


In [ ]:
# Inicializar la sesión de Spark
spark = init_spark()


In [ ]:
# Cargar los datasets desde los archivos JSON
agencies_df = load_json_data(spark, "path_to_your_data/agencies_file.json")
brands_df = load_json_data(spark, "path_to_your_data/brands_file.json")
cars_df = load_json_data(spark, "path_to_your_data/cars_file.json")
customers_df = load_json_data(spark, "path_to_your_data/customers_file.json")
rentals_df = load_json_data(spark, "path_to_your_data/rentals_file.json")


In [ ]:
from pyspark.sql import functions as F

# Extraer información de las columnas JSON
agencies_df = agencies_df.withColumn('agency_name', F.get_json_object('agency_info', '$.agency_name'))\
                         .withColumn('city', F.get_json_object('agency_info', '$.city'))

brands_df = brands_df.withColumn('brand_name', F.get_json_object('brand_info', '$.brand_name'))\
                     .withColumn('country', F.get_json_object('brand_info', '$.country'))

cars_df = cars_df.withColumn('car_name', F.get_json_object('car_info', '$.car_name'))

customers_df = customers_df.withColumn('customer_name', F.get_json_object('customer_info', '$.customer_name'))


In [ ]:
# Unir los dataframes basados en las relaciones entre ellos
final_df = rentals_df.join(agencies_df, rentals_df.agency_id == agencies_df.agency_id) \
                     .join(cars_df, rentals_df.car_id == cars_df.car_id) \
                     .join(customers_df, rentals_df.customer_id == customers_df.customer_id) \
                     .join(brands_df, cars_df.car_id == brands_df.car_id)

# Seleccionar las columnas necesarias para el dataframe final
final_df = final_df.select('rental_id', 'car_name', 'agency_name', 'customer_name')


In [ ]:
# Mostrar el dataframe final
final_df.show()
